# HICSS 2024: A Restriction Wrapper for Multi-Agent RL Libraries

This notebook demonstrates the basic functionality of the restriction wrapper described in _M. Oesterle and T. Grams (2024): A Restriction Wrapper for Multi-Agent RL Libraries_. More detailed examples for both _PettingZoo_ and _RLlib_ wrappers can be found in the respective notebooks at `./examples/`.

In [1]:
from src.utils import test_environment
from examples.envs.rllib import DummyEnvironment

In [2]:
test_environment(DummyEnvironment)

Testing <class 'examples.envs.rllib.DummyEnvironment'>...
Reset: {'0': array([0.19446187, 0.33879334], dtype=float32), '1': array([0.04443369, 0.9029297 ], dtype=float32)}
Actions: {'0': 1, '1': 0}
Step:  {'0': array([0.14841695, 0.0936696 ], dtype=float32), '1': array([0.62695646, 0.13435353], dtype=float32)}, {'0': 0.241476638264979, '1': 0.7079546031007848}, {'__all__': False}
Actions: {'0': 0, '1': 0}
Step:  {'0': array([0.90185905, 0.2663394 ], dtype=float32), '1': array([0.3961667 , 0.01047942], dtype=float32)}, {'0': 0.9989765254186834, '1': 0.12687284776423868}, {'__all__': False}
Actions: {'0': 0, '1': 2}
Step:  {'0': array([0.9589378, 0.9024878], dtype=float32), '1': array([0.182821 , 0.0940948], dtype=float32)}, {'0': 0.4495709708789719, '1': 0.8787147245115562}, {'__all__': False}
Actions: {'0': 2, '1': 2}
Step:  {'0': array([0.26544398, 0.7768176 ], dtype=float32), '1': array([0.16942886, 0.21606648], dtype=float32)}, {'0': 0.4757041131230887, '1': 0.8706940755305589}, {'_